# Em busca da soma pythônica

Na [Python-list](https://fpy.li/12-11), há um fio de abril de 2003 com
o título
[_Pythonic Way to Sum n-th List Element?_](https://fpy.li/12-12)
(A forma pythônica de somar o n-ésimo elemento em listas).

Não há uma resposta única para a "O que é pythônico?", da mesma
forma que não há uma resposta única para "O que é belo?".

Mas talvez esta discussão dê alguma luz.

O autor original do fio, Guy Middleton, pediu melhorias para a solução abaixo,
afirmando não gostar de usar `lambda`.

No caso específico, ele quer somar o segundo item de cada lista de uma série de listas.

In [1]:
from functools import reduce
my_list = [[2, 3, 4], [22, 33, 44], [7, 6, 5]]
reduce(lambda ac, n: ac+n, [sub[1] for sub in my_list])

42

Este código usa um excesso de elementos idiomáticos de Python: `lambda`, `reduce` e uma compreensão de lista.

## Sobre o reduce

Esta é assinatura de `reduce`:

```python
def reduce(function, iterable, /, initial=initial_missing): ...
```

`function` tem que ser uma função que aceita dois argumentos: um acumulador, e um item.
O acumulador é inicializado com `initial`, ou com o primeiro item do `iterable`.
A função será invocada uma vez para cada item do iterável,
recebendo o acumulador e o próximo item. O resultado serve para atualizar o acumulador
antes da próxima chamada.

Uma versão simplificada de `reduce`, lidando só com sequências, já serve para ilustrar o algoritmo:

In [2]:
def reduzir(função, sequência):
    acumulador = sequência[0]
    for item in sequência[1:]:
        acumulador = função(acumulador, item)
    return acumulador

Por exemplo, o fatorial de 5 pode ser calculado assim, sem usar `lambda`:

In [3]:
def mult(ac, n):
    return ac * n
    
reduzir(mult, range(2, 6))

120

(Implementar `reduzir` com suporte a iteráveis e o argumento `inicial` é um exercício legal.)

## Voltando ao problema de somar o n-ésimo item

A solução que Guy Middleton apresentou
provavelmente ficaria em último lugar em um concurso de popularidade, pois
ofende quem odeia `lambda` e também aqueles que desprezam as compreensões de
lista.

Nem o próprio Guy curtiu o código que ele apresentou:

In [4]:
from functools import reduce
my_list = [[2, 3, 4], [22, 33, 44], [7, 6, 5]]
reduce(lambda ac, n: ac+n, [sub[1] for sub in my_list])

42

Se você vai usar `lambda`, provavelmente não há razão para usar uma compreensão
de lista—exceto para filtragem, que não é o caso aqui.

Aqui está uma solução minha que ofenderá todo mundo, exceto os amantes de `lambda`:

In [5]:
reduce(lambda ac, sub: ac + sub[1], my_list, 0)

42

Não participei da discussão original, e não usaria o trecho acima em código real,
pois eu também não gosto muito de `lambda`.

Apenas quis mostrar um exemplo sem uma compreensão de lista.

A primeira resposta veio de Fernando Perez, criador do IPython,
mostrando como a NumPy suporta arrays _n_-dimensionais e fatiamento _n_-dimensional:

In [6]:
import numpy as np
my_array = np.array(my_list)
np.sum(my_array[:, 1])

np.int64(42)

Acho a solução de Perez boa, mas Guy Middleton elegiou essa próxima solução,
de Paul Rubin e Skip Montanaro:

In [7]:
import operator
reduce(operator.add, [sub[1] for sub in my_list], 0)

42

Então Evan Simpson perguntou, "O que há de errado em fazer assim?":

In [8]:
ac = 0
for sub in my_list:
    ac += sub[1]

ac

42

Muitos concordaram que esse código era bastante pythônico.
Alex Martelli chegou a escrever que Guido provavelmente 
resolveria o problema desta maneira.

Gosto do código de Evan Simpson, mas também gosto do comentário
de David Eppstein sobre ele:

> Se você quer a soma de uma lista de itens, deveria escrever algo como
"a soma de uma lista de itens", não como "faça um loop sobre
esses itens, mantenha uma variável `ac`, execute uma série de somas".
Por que temos linguagens de alto nível, senão para expressar nossas
intenções em um nível mais alto e deixar a linguagem se preocupar
com as operações de baixo nível necessárias para executá-las?

E daí Alex Martelli voltou para sugerir:

> Fazemos somas com tanta frequência que eu não me importaria de forma
alguma se Python a tornasse uma função embutida. Mas `reduce(operator.add,
...)` não é uma boa forma de expressar isso, na minha opinião (e vejam
que, como um antigo *APLer* e *FP-liker* eu _deveria_ gostar daquilo, mas não gosto).

(Aqui o Martelli faz referência às linguagens funcionas
[APL](https://pt.wikipedia.org/wiki/APL_(linguagem_de_programa%C3%A7%C3%A3o)) e
[FP](https://pt.wikipedia.org/wiki/FP_(linguagem_de_programa%C3%A7%C3%A3o)))

## A soma pythônica

Martelli então sugere uma função `sum()`, que ele mesmo programa e submete para
o Python.

Ela se torna uma função embutida no Python 2.3, lançado apenas três
meses após aquela conversa na lista.

A sintaxe preferida de Martelli se torna a regra:

In [9]:
sum([sub[1] for sub in my_list])

42

No final do ano seguinte (novembro de 2004), Python 2.4 foi lançado e incluía
expressões geradoras, fornecendo o que agora é, na minha opinião, a resposta
mais pythônica para a pergunta original de Guy Middleton:

In [10]:
sum(sub[1] for sub in my_list)

42

Isso não só é mais legível que `reduce`,
também trata o problema da sequência
vazia, porque `sum([])` é `0`.

Na mesma conversa, Alex Martelli sugeriu que a função embutida `reduce` de
Python 2 trazia mais problemas que soluções, porque encorajava idiomas de
programação difíceis de explicar. Ele foi bastante convincente: a função foi
rebaixada para o módulo `functools` no Python 3.

Ainda assim, `functools.reduce` tem seus usos.
Mas quando pensar em `reduce`,
veja se `sum`, `all` e `any` resolvem seu problema
mais facilmente.

----

Este notebook foi adaptado da seção **Ponto de Vista** do
[Capítulo 12](https://pythonfluente.com/2/#_para_saber_mais_5)
do **Python Fluente, 2ª ed.**